In [5]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141164
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-141164


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
udacity_compute_name = 'udacityproj1comp'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)

compute_target = ComputeTarget.create(workspace=ws, 
name=udacity_compute_name, provisioning_configuration=compute_config)

compute_target.wait_for_completion(show_output=True)


SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [36]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.script_run_config import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {'--C': uniform(0.0,1.0),
    '--max_iter':choice(10,100,500)}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
compute_target = compute_target,
entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps, policy=policy, primary_metric_name="Accuracy", 
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=4)

In [37]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
from azureml.pipeline.steps import HyperDriveStep
from azureml.pipeline.core import Pipeline

# Alternatively, run with pipeline
#trainWithHyperDriveStep =  HyperDriveStep(name='HyperDrive_Classification', 
#hyperdrive_config=hyperdrive_config, 
#estimator_entry_script_arguments=[]
#)
#pipeline_steps = [trainWithHyperDriveStep]
#hyperdrive_pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
#pipeline_run = exp.submit(hyperdrive_pipeline, regenerate_outputs=False)

# Run without the pipeline, but passing the config file.
run = exp.submit(hyperdrive_config, show_output=True)


In [38]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
run.wait_for_completion()

best_run = run.get_best_run_by_primary_metric()
joblib.dump(best_run, 'best_run_hd.joblib')
print(best_run)

TypeError: cannot pickle '_queue.SimpleQueue' object

In [56]:
import pandas as pd
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
pd.Series(best_parameters, name='Optimal Parameters').to_frame()

,Value
--C,0.4328762174558086
--max_iter,500


In [53]:
print(best_run)

Run(Experiment: udacity-project,
Id: HD_86854fb2-9f43-4689-9aa3-9e3bd00a4b85_18,
Type: azureml.scriptrun,
Status: Completed)


In [54]:
best_run.get_metrics()

{'Regularization Strength:': 0.4328762174558086,
 'Max iterations:': 500,
 'Accuracy': 0.9110722825087364}

In [49]:
joblib.dump(best_run, 'best_run_hd.joblib')
print(best_run)

TypeError: 'Run' object is not subscriptable

In [58]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

print(ds)

TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
x['y'] = y

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=4)

In [74]:
import azureml.train.automl
azureml.train.automl.VERSION

'1.24.0'

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
run_auto_ml = exp.submit(config=automl_config, show_output=True)


No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_d499bf16-395d-419f-9aba-289934d7ecf1

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [14]:
# Retrieve and save your best automl model.


### YOUR CODE HERE ###

best_run_automl = run_auto_ml.get_best_run_by_primary_metric()


AttributeError: 'AutoMLRun' object has no attribute 'get_best_run_by_primary_metric'

In [28]:
[child for child in run_auto_ml.get_children()][1]

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d499bf16-395d-419f-9aba-289934d7ecf1_42,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [23]:
run_auto_ml.get_metrics()

{'AUC_micro': 0.9812344296712543,
 'AUC_macro': 0.9492836807266527,
 'weighted_accuracy': 0.9595291588997443,
 'precision_score_micro': 0.9170560108945351,
 'log_loss': 0.2265210050557988,
 'precision_score_macro': 0.8032554677075618,
 'f1_score_micro': 0.9170560108945351,
 'accuracy': 0.9170560108945351,
 'norm_macro_recall': 0.49212634282090645,
 'recall_score_macro': 0.7460631714104533,
 'recall_score_micro': 0.9170560108945351,
 'f1_score_macro': 0.7703762261973794,
 'AUC_weighted': 0.9492836807266527,
 'average_precision_score_micro': 0.9820393537196008,
 'precision_score_weighted': 0.9106658782625752,
 'average_precision_score_weighted': 0.9566514492523845,
 'balanced_accuracy': 0.7460631714104533,
 'matthews_correlation': 0.5462282922085844,
 'average_precision_score_macro': 0.8291897421863088,
 'recall_score_weighted': 0.9170560108945351,
 'f1_score_weighted': 0.9127741258425033}

In [26]:
import joblib

joblib.dump(best_run_automl, 'best_run_automl.pkl')


TypeError: can't pickle _thread.RLock objects

In [1]:

# Proof of compute_target.delete()
compute_target.delete()

NameError: name 'compute_target' is not defined